In [1]:
import numpy as np
import pandas as pd

# Project 1

## Data

I'm using data from two sources. The first is FiveThirtyEight's [Food World Cup](https://fivethirtyeight.com/features/the-fivethirtyeight-international-food-associations-2014-world-cup/). The second is a the World Bank's most popular [World Development Indicators](https://databank.worldbank.org/indicator/NY.GDP.MKTP.CD/1ff4a498/Popular-Indicators). 


In [44]:
foodwc = pd.read_csv('data/processed/food-world-cup-data2.csv')
worldbankdata = pd.read_csv('data/processed/world_bank_data.csv')


Looking at the Food World Cup (FWC) data, we see that it indexes on an identifier for respondents to the surveys used by FiveThirtyEight. The other columns are the rating each respondent (Americans  in this case) has for each country's cuisine, and some ancillary data about the respondent's interest in cuisines, income, age, etc. 

In [45]:
foodwc.columns

Index(['RespondentID',
       'Generally speaking, how would you rate your level of knowledgeof cuisines from different parts of the world?',
       'How much, if at all, are you interested in cuisines from different parts of the world?',
       'Please rate how much you like the traditional cuisine of Algeria:',
       'Please rate how much you like the traditional cuisine of Argentina.',
       'Please rate how much you like the traditional cuisine ofAustralia.',
       'Please rate how much you like the traditional cuisine of Belgium.',
       'Please rate how much you like the traditional cuisine of Bosnia and Herzegovina.',
       'Please rate how much you like the traditional cuisine of Brazil.',
       'Please rate how much you like the traditional cuisine of Cameroon.',
       'Please rate how much you like the traditional cuisine of Chile.',
       'Please rate how much you like the traditional cuisine of Colombia.',
       'Please rate how much you like the traditional cuisin

We can see that the countries' rating columns are in the form of a statement. We want to remove the statement to leave us with only the country name. We can also simplify some of the longer column names. We also had some problems with encoding the original data into utf-8 and some spaces are missing in columns 1 and 5, but we can program around this. 

In [46]:
prefix = 'Please rate how much you like the traditional cuisine of'
prefix_len = len(prefix)

new_columns = []

for col in foodwc.columns:
    if col.startswith(prefix):
        new_columns.append(col[prefix_len:-1].strip())
    else:
        new_columns.append(col)
        
new_columns

new_columns[1] = 'Cuisine Knowledge'
new_columns[2] = 'Cuisine Interest'
new_columns[-1] = 'Locations'
new_columns[26] = 'Netherlands'

One thing to note is that the World Bank Data refers to 'the Netherlands' as simply 'Netherlands' so we make that change here as well.

In [47]:
foodwc.columns = new_columns

The data from the World Bank is much more complex. The 'Series Name' column names various metrics that we might want. We want to split it into different tables for different metrics indexed on country name. For this initial project we chose GDP and population as the metrics we want to combine with the FWC data and we'll use the year 2014 since that's when the FWC article was released. 

In [48]:
worldbankdata.columns

Index(['Series Name', 'Series Code', 'Country Name', 'Country Code',
       '2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]', '2003 [YR2003]',
       '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]', '2007 [YR2007]',
       '2008 [YR2008]', '2009 [YR2009]', '2010 [YR2010]', '2011 [YR2011]',
       '2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]', '2015 [YR2015]'],
      dtype='object')

In [49]:
wbd2014 = worldbankdata.loc[:, ['Series Name', 'Country Name', '2014 [YR2014]']]
wbd2014["Series Name"].astype("category").cat.categories

Index(['Adolescent fertility rate (births per 1,000 women ages 15-19)',
       'Agriculture, forestry, and fishing, value added (% of GDP)',
       'Births attended by skilled health staff (% of total)',
       'CO2 emissions (metric tons per capita)',
       'Contraceptive prevalence, any methods (% of women ages 15-49)',
       'Data from database: World Development Indicators',
       'Electric power consumption (kWh per capita)',
       'Energy use (kg of oil equivalent per capita)',
       'Exports of goods and services (% of GDP)',
       'External debt stocks, total (DOD, current US$)',
       'Fertility rate, total (births per woman)',
       'Foreign direct investment, net (BoP, current US$)',
       'Foreign direct investment, net inflows (BoP, current US$)',
       'Forest area (sq. km)', 'GDP (current US$)', 'GDP growth (annual %)',
       'GDP per capita (current US$)',
       'GNI per capita, Atlas method (current US$)',
       'GNI per capita, PPP (current international 

In [50]:
wbdgdp = wbd2014[wbd2014['Series Name'] == 'GDP per capita (current US$)'].loc[:, ['Country Name', '2014 [YR2014]']]
wbdpop = wbd2014[wbd2014['Series Name'] == 'Population, total'].loc[:, ['Country Name', '2014 [YR2014]']]

wbdgdp = wbdgdp.set_index('Country Name')
wbdpop = wbdpop.set_index('Country Name')

wbdgdp = wbdgdp.rename(columns={'2014 [YR2014]': 'GDP per Capita'})
wbdpop = wbdpop.rename(columns={'2014 [YR2014]': 'Total Population'})

wbdfinal = pd.merge(wbdgdp, wbdpop, on='Country Name')

So we now have 2 clean tables. To combine them, we take the mean of the ratings in the FWC data and join those to the World Bank metrics. We need to cut some columns from the FWC data to work only with Country Names. We start by looking at which columns there are. 

In [51]:
foodwc.columns

Index(['RespondentID', 'Cuisine Knowledge', 'Cuisine Interest', 'Algeria',
       'Argentina', 'Australia', 'Belgium', 'Bosnia and Herzegovina', 'Brazil',
       'Cameroon', 'Chile', 'Colombia', 'Costa Rica', 'Croatia', 'Ecuador',
       'England', 'France', 'Germany', 'Ghana', 'Greece', 'Honduras', 'Iran',
       'Italy', 'Ivory Coast', 'Japan', 'Mexico', 'Netherlands', 'Nigeria',
       'Portugal', 'Russia', 'South Korea', 'Spain', 'Switzerland',
       'United States', 'Uruguay', 'China', 'India', 'Thailand', 'Turkey',
       'Cuba', 'Ethiopia', 'Vietnam', 'Ireland', 'Gender', 'Age',
       'Household Income', 'Education', 'Locations'],
      dtype='object')

We then cut non-country names from the DataFrame and then find the mean before formatting the data so it joins cleanly with the cleaned up World Bank Data. 

In [52]:
foodwc2 = foodwc.iloc[:, 3:-5]
foodwc3 = foodwc2.mean()
foodwc4 = pd.DataFrame({'Country Name':foodwc3.index, 'Average Rating':foodwc3.values})

We now have the final table relating the average cuisine rating with the GDP per capita and population of the countries.

In [53]:
final_table = pd.merge(foodwc4, wbdfinal, on='Country Name')
final_table

,Country Name,Average Rating,GDP per Capita,Total Population
0,Algeria,3.333333,5493.02558996263,38923687
1,Argentina,3.558140,12334.7982453893,42669500
2,Australia,3.196429,62510.7911705641,23475686
3,Belgium,3.543478,47700.5403601178,11209057
4,Bosnia and Herzegovina,3.153846,5329.63504488766,3482104
5,Brazil,3.644068,12112.5882059697,202763735
6,Cameroon,3.400000,1542.62026621531,22681858
7,Chile,3.475000,14670.996760602,17758959
8,Colombia,3.250000,8114.08406910545,46969209
9,Costa Rica,3.568182,10547.1518593556,4795396


From this final table of data, we could look for correlations between Ratings and GDP per capita or population. 